<a href="https://colab.research.google.com/github/ViniViniAntunes/Analise-descritiva-PNAD-2015_Parte-2/blob/master/Projeto_Final_Estimativas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# CURSO DE ESTATÍSTICA - PARTE 2
***

## Trabalho sobre Probabilidades, Amostragem e Estimações

Utilizando os conhecimentos adquiridos em nosso treinamento execute as tarefas abaixo. Siga o roteiro proposto e vá completando as células vazias.

# DATASET DO PROJETO
***

### Pesquisa Nacional por Amostra de Domicílios - 2015

A <b>Pesquisa Nacional por Amostra de Domicílios - PNAD</b> investiga anualmente, de forma permanente, características gerais da população, de educação, trabalho, rendimento e habitação e outras, com periodicidade variável, de acordo com as necessidades de informação para o país, como as características sobre migração, fecundidade, nupcialidade, saúde, segurança alimentar, entre outros temas. O levantamento dessas estatísticas constitui, ao longo dos 49 anos de realização da pesquisa, um importante instrumento para formulação, validação e avaliação de políticas orientadas para o desenvolvimento socioeconômico e a melhoria das condições de vida no Brasil.

### Fonte dos Dados

https://ww2.ibge.gov.br/home/estatistica/populacao/trabalhoerendimento/pnad2015/microdados.shtm

### Variáveis utilizadas

> ### Renda
> ***

Rendimento mensal do trabalho principal para pessoas de 10 anos ou mais de idade.

> ### Idade
> ***

Idade do morador na data de referência em anos.

> ### Altura (elaboração própria)
> ***

Altura do morador em metros.

> ### UF
> ***

|Código|Descrição|
|---|---|
|11|Rondônia|
|12|Acre|
|13|Amazonas|
|14|Roraima|
|15|Pará|
|16|Amapá|
|17|Tocantins|
|21|Maranhão|
|22|Piauí|
|23|Ceará|
|24|Rio Grande do Norte|
|25|Paraíba|
|26|Pernambuco|
|27|Alagoas|
|28|Sergipe|
|29|Bahia|
|31|Minas Gerais|
|32|Espírito Santo|
|33|Rio de Janeiro|
|35|São Paulo|
|41|Paraná|
|42|Santa Catarina|
|43|Rio Grande do Sul|
|50|Mato Grosso do Sul|
|51|Mato Grosso|
|52|Goiás|
|53|Distrito Federal|

> ### Sexo	
> ***

|Código|Descrição|
|---|---|
|0|Masculino|
|1|Feminino|

> ### Anos de Estudo
> ***

|Código|Descrição|
|---|---|
|1|Sem instrução e menos de 1 ano|
|2|1 ano|
|3|2 anos|
|4|3 anos|
|5|4 anos|
|6|5 anos|
|7|6 anos|
|8|7 anos|
|9|8 anos|
|10|9 anos|
|11|10 anos|
|12|11 anos|
|13|12 anos|
|14|13 anos|
|15|14 anos|
|16|15 anos ou mais|
|17|Não determinados| 
||Não aplicável|

> ### Cor
> ***

|Código|Descrição|
|---|---|
|0|Indígena|
|2|Branca|
|4|Preta|
|6|Amarela|
|8|Parda|
|9|Sem declaração|

#### Observação
***
> Os seguintes tratamentos foram realizados nos dados originais:
> 1. Foram eliminados os registros onde a <b>Renda</b> era inválida (999 999 999 999);
> 2. Foram eliminados os registros onde a <b>Renda</b> era missing;
> 3. Foram considerados somente os registros das <b>Pessoas de Referência</b> de cada domicílio (responsável pelo domicílio).

***
***

### Utilize a célula abaixo para importar as bibliotecas que precisar para executar as tarefas

In [0]:
import pandas as pd
import numpy as np
from scipy.special import comb
from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import norm

### Importe o dataset e armazene o conteúdo em uma DataFrame

In [0]:
dados = pd.read_csv('dados.csv', sep = ',')

### Visualize o conteúdo do DataFrame

In [4]:
dados.head()

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura
0,11,0,23,8,12,800,1.603808
1,11,1,23,2,12,1150,1.739790
2,11,1,35,8,15,880,1.760444
3,11,0,46,2,6,3500,1.783158
4,11,1,47,8,9,150,1.690631


# Problema A

Avaliando nosso dataset é possível verificar que a **proporção de homens** como chefes de domicílios é de quase **70%**.
#### Como parte da tarefa, verifique a real proporção de homens e mulheres em nosso dataset (vimos como fazer isso em nosso primeiro curso de estatística).

In [5]:
sexo = {0: 'Masculino', 1: 'Feminino'}
freq_sexo = dados.Sexo.value_counts()
percent_sexo = dados.Sexo.value_counts(normalize = True) * 100
data_frame = {'Frequência': freq_sexo, 'Porcentagem (%)': percent_sexo}
dist_freq_sexo = pd.DataFrame(data_frame)
dist_freq_sexo.rename_axis('Sexo', axis = 'columns', inplace = True)
dist_freq_sexo.rename(index = sexo, inplace = True)
dist_freq_sexo

Sexo,Frequência,Porcentagem (%)
Masculino,53250,69.299844
Feminino,23590,30.700156


Precisamos **selecionar aleatoriamente grupos de 10 indivíduos** para verificar as diferenças entre os rendimentos em cada grupo. Qual a **probabilidade de selecionamos um grupo que apresente a mesma proporção da população**, ou seja, selecionarmos um grupo que seja **composto por 7 homens e 3 mulheres**?

#### Verifique que tipo de distribuição de probabilidade se encaixa neste experimento.

Resp: O tipo de distribuição de probabilidade que melhor se encaixa nesse caso, é a distribuição binomial, já que só existe duas únicas possibilidades de respostas (sexo maisculino e feminino) e que são excludentes (se é do sexo masculino, não é do sexo feminino).

### Solução

In [6]:
# Probabilidade de se escolher um homem ==> p
p = 69.299844 / 100
p

# Probabilidade de se escolher mulher ==> q
#q = 1- p

0.6929984399999999

In [7]:
# Número de eventos (selecionar 10 indivíduos) ==> n
n = 10
n

10

In [8]:
# Número de eventos de sucesso (dentre esse 10, escolher 7 homens) ==> k
k = 7
k

7

In [9]:
# Cálculo da probabilidade pedida
probabilidade = binom.pmf(k, n, p)
print("A probabilidade de selecionamos um grupo que apresente a mesma \n"
      "proporção da população, ou seja, selecionarmos um grupo que seja \n"
      "composto por 7 homens e 3 mulheres é de {:.2f}%".format(probabilidade * 100))

A probabilidade de selecionamos um grupo que apresente a mesma 
proporção da população, ou seja, selecionarmos um grupo que seja 
composto por 7 homens e 3 mulheres é de 26.65%


#Problema B

Ainda sobre a questão anterior, **quantos grupos de 10 indivíduos** nós precisaríamos selecionar, de forma aleatória, para conseguir **100 grupos compostos por 7 homens e 3 mulheres**?

#### Lembre-se da forma de cálculo da média de uma distribuição binomial

### Solução

In [10]:
# Probabilidade de selecionar os tais grupos
p = probabilidade
p

0.2665193784779078

In [11]:
# Média binomial
media = 100
media

100

In [12]:
# Número de grupos de 10 indivíduos que precisaríamos selecionar
# media = n * p
n = media / p
print("Para conseguir 100 grupos compostos por 7 homens e 3 mulheres \n"
      "precisaríamos selecionar, de forma aleatória, {} grupos \n"
      "de 10 indivíduos.".format(int(n.round())))

Para conseguir 100 grupos compostos por 7 homens e 3 mulheres 
precisaríamos selecionar, de forma aleatória, 375 grupos 
de 10 indivíduos.


# Problema C

Um cliente nos encomendou um estudo para avaliar o **rendimento dos chefes de domicílio no Brasil**. Para isso precisamos realizar uma nova coleta de dados, isto é, uma nova pesquisa de campo. Após reunião com o cliente foi possível elencar o seguinte conjunto de informações:

> A. O resultado da pesquisa precisa estar pronto em **2 meses**;

> B. Teremos somente **R$\$$ 150.000,00** de recursos para realização da pesquisa de campo; e
    
> C. Seria interessante uma **margem de erro não superior a 10% em relação a média estimada**.

Em nossa experiência com estudos deste tipo, sabemos que o **custo médio por indivíduo entrevistado fica em torno de R$\$$ 100,00**. Com este conjunto de fatos avalie e obtenha o seguinte conjunto de informações para passar ao cliente:


> 1. Para obter uma estimativa para os parâmetros da população (renda dos chefes de domicílio no Brasil), realize uma amostragem aleatória simples em nosso conjunto de dados. Essa amostra deve conter 200 elementos (utilize random_state = 101 para garantir que o mesmo experimento posso ser realizado novamente). Obtenha a média e o desvio-padrão dessa amostra.
    
> 2. Para a **margem de erro** especificada pelo cliente obtenha os **tamanhos de amostra** necessários para garantir os **níveis de confiança de 90%, 95% e 99%**.
    
> 3. Obtenha o **custo da pesquisa** para os três níveis de confiança.
    
> 4. Para o maior nível de confiança viável (dentro do orçamento disponível), obtenha um **intervalo de confiança para a média da população**.
    
> 5. Assumindo o **nível de confiança escolhido no item anterior**, qual **margem de erro** pode ser considerada utilizando todo o recurso disponibilizado pelo cliente?
    
> 6. Assumindo um **nível de confiança de 95%**, **quanto a pesquisa custaria ao cliente** caso fosse considerada uma **margem de erro de apenas 5%** em relação a média estimada?


#Solução do item 1</font>

### Seleção de uma amostra aleatório simples

#### Lembre-se de utilizar *random_state = 101*

In [13]:
amostra = dados.sample(n = 200, random_state = 101)
amostra.head()

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura
29042,29,0,39,8,5,480,1.719128
62672,43,0,55,2,6,250,1.639205
29973,29,1,36,2,12,788,1.654122
22428,26,0,46,8,8,1680,1.622450
55145,41,0,37,2,9,2500,1.625268


In [14]:
media_amostra = amostra.Renda.mean()
print("A média de renda da amostra é de R${:.2f}".format(media_amostra))

A média de renda da amostra é de R$1964.20


In [15]:
desvio_padrao_amostra = amostra.Renda.std()
print("A desvio padrão da renda da amostra"
      " é de R${:.2f}".format(desvio_padrao_amostra))

A desvio padrão da renda da amostra é de R$3139.89


#Solução do item 2

### Dados do problema

In [16]:
# Média
print("R${:.2f}".format(media_amostra))

R$1964.20


In [17]:
# Desvio padrão
print("R${:.2f}".format(desvio_padrao_amostra))

R$3139.89


### Obtenha a margem de erro

#### Lembre-se que a margem de erro deve estar na mesma unidade da variável que está sendo estudada (R$)

In [18]:
# Margem de erro de 10%, em R$
e = 0.10 * media_amostra
print("A margem de erro é de R${:.2f} para mais ou para menos.".format(e))

A margem de erro é de R$196.42 para mais ou para menos.


### Tamanho da amostra ($1 - \alpha = 90\%$)

In [19]:
# Obtendo o valor de z para um nível de confiança de 90%
z_90 = norm.ppf(0.5 + 0.90 / 2)
z_90

1.6448536269514722

In [20]:
# Calculando o tamanho na amostra para esse nível de confiança
n_90 = (z_90 * (desvio_padrao_amostra / e)) ** 2
n_90 = int(n_90.round())
print("O tamanho da amostra para um nível de \n"
      "confiança de 90% é de {}".format(n_90))

O tamanho da amostra para um nível de 
confiança de 90% é de 691


### Tamanho da amostra ($1 - \alpha = 95\%$)

In [21]:
# Obtendo o valor de z para um nível de confiança de 95%
z_95 = norm.ppf(0.5 + 0.95 / 2)
z_95

1.959963984540054

In [22]:
# Calculando o tamanho na amostra para esse nível de confiança
n_95 = (z_95 * (desvio_padrao_amostra / e)) ** 2
n_95 = int(n_95.round())
print("O tamanho da amostra para um nível de \n"
      "confiança de 90% é de {}".format(n_95))

O tamanho da amostra para um nível de 
confiança de 90% é de 982


### Tamanho da amostra ($1 - \alpha = 99\%$)

In [23]:
# Obtendo o valor de z para um nível de confiança de 99%
z_99 = norm.ppf(0.5 + 0.99 / 2)
z_99

2.5758293035489004

In [24]:
# Calculando o tamanho na amostra para esse nível de confiança
n_99 = (z_99 * (desvio_padrao_amostra / e)) ** 2
n_99 = int(n_99.round())
print("O tamanho da amostra para um nível de \n"
      "confiança de 90% é de {}".format(n_99))

O tamanho da amostra para um nível de 
confiança de 90% é de 1695


# Solução do item 3

Segundo o enunciado do problema, o custo médio por indivíduo entrevistado fica em torno de R $  100,00.

Então:

In [0]:
custo_individual = 100

### Custo da pesquisa para o nível de confiança de 90%

In [26]:
custo_90 = n_90 * custo_individual
print("Para um nível de confiança de 90%, \n"
      "o custo da pesquisa é de R${:,.2f}".format(custo_90))

Para um nível de confiança de 90%, 
o custo da pesquisa é de R$69,100.00


### Custo da pesquisa para o nível de confiança de 95%

In [27]:
custo_95 = n_95 * custo_individual
print("Para um nível de confiança de 95%, \n"
      "o custo da pesquisa é de R${:,.2f}".format(custo_95))

Para um nível de confiança de 95%, 
o custo da pesquisa é de R$98,200.00


### Custo da pesquisa para o nível de confiança de 99%

In [28]:
custo_99 = n_99 * custo_individual
print("Para um nível de confiança de 99%, \n"
      "o custo da pesquisa é de R${:,.2f}".format(custo_99))

Para um nível de confiança de 99%, 
o custo da pesquisa é de R$169,500.00


# Solução do item 4

In [29]:
intervalo = norm.interval(alpha = 0.95, loc = media_amostra, scale = desvio_padrao_amostra / np.sqrt(n_95))
print("O intervalo de confiança é \n"
      "de R${}\n"
      "até R${}".format(round(intervalo[0], 2), round(intervalo[1], 2)))

O intervalo de confiança é 
de R$1767.82
até R$2160.59


# Solução do item 5

In [0]:
# Recurso estipulado pelo enunciado
recursos = 150000

In [31]:
# Descobrindo o maior tamanho da amostra limitado pelo recurso de R$150.000,00
n_max = recursos / custo_individual
n_max

1500.0

In [32]:
# Calculando a margem de erro que pode ser considerada utilizando 
#   todo o recurso disponibilizado
e_min = z_95 * (desvio_padrao_amostra / np.sqrt(n_max))
print("A margem de erro que pode ser considerada utilizando \n"
      "todo o recurso é de R${:.2f}".format(e_min))

A margem de erro que pode ser considerada utilizando 
todo o recurso é de R$158.90


# Solução do item 6

In [33]:
# Margem de erro de 5%, em R$
e_5 = 0.05 * media_amostra
print("A margem de erro é de R${:.2f} para mais ou para menos.".format(e_5))

A margem de erro é de R$98.21 para mais ou para menos.


In [34]:
# Calculando o tamanho na amostra para o nível de confiança de 95% 
#   e margem de erro de 5% em relação à média
n_95_e_5 = (z_95 * (desvio_padrao_amostra / e_5)) ** 2
n_95_e_5 = int(n_95_e_5.round())
print("O tamanho da amostra para um nível de \n"
      "confiança de 90% é de {}".format(n_95_e_5))

O tamanho da amostra para um nível de 
confiança de 90% é de 3927


In [0]:
# Calculando o custo da pesquisa para um nível de confiança de 95% 
#   e uma margem de erro de 5% em relação à média
custo_95_e_5 = n_95_e_5 * custo_individual

In [36]:
print("O custo da pesquisa seria de R${:,.2f}".format(custo_95_e_5))

O custo da pesquisa seria de R$392,700.00
